<a href="https://colab.research.google.com/github/Saransh1329/BlackBox-Agentic-AI-for-Predictive-Maintenance/blob/main/ai2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
import io

# ========================
# STEP 1: UPLOAD FILE
# ========================
print("Upload your Excel file:")
uploaded = files.upload()
filename = next(iter(uploaded.keys()))
xls = pd.ExcelFile(io.BytesIO(uploaded[filename]))

failure_df = pd.read_excel(xls, "Failiure Prediction")
owner_df   = pd.read_excel(xls, "Owner Details")
appt_df    = pd.read_excel(xls, "Appointment")
maint_df   = pd.read_excel(xls, "Maintainence Records")

# ========================
# STEP 2: CONFIG
# ========================
FAILURE_PROB_COL = "pred_failure_prob_pct"
FAILURE_VEHICLE_ID_COL = "vehicle_id"

OWNER_SERVICE_CENTER_COL = "assigned_service_center"
OWNER_VEHICLE_ID_COL = "vehicle_id"

APPT_SERVICE_CENTER_COL = "service_center"
APPT_DATE_COL = "date"
APPT_TIME_COL = "slot_time"

TECHNICIAN_COLS = [
    "Technician_1","Technician_2","Technician_3",
    "Technician_4","Technician_5","Technician_6"
]

FREE_VALUE = "free"

# REQUIRED NEW COLUMNS
NEW_DATE_COL = "assigned_slot_date"
NEW_TIME_COL = "assigned_slot_time"
NEW_TECH_COL = "assigned_technician"

# ensure new columns exist in owner_df
for c in [NEW_DATE_COL, NEW_TIME_COL, NEW_TECH_COL]:
    if c not in owner_df.columns:
        owner_df[c] = ""


# ========================
# STEP 3: VEHICLES TO PROCESS
# ========================
if FAILURE_PROB_COL not in failure_df.columns:
    raise KeyError(
        f"Missing '{FAILURE_PROB_COL}' in Failiure Prediction sheet.\n"
        f"Available columns: {list(failure_df.columns)}"
    )

at_risk = failure_df[failure_df[FAILURE_PROB_COL] > 80]
vehicle_ids = at_risk[FAILURE_VEHICLE_ID_COL].unique()

appt_df[APPT_DATE_COL] = pd.to_datetime(appt_df[APPT_DATE_COL], errors="coerce")

# ========================
# STEP 3.5: TRACKING ASSIGNED SLOTS
# ========================
# Create a set to track assigned slots: (service_center, date, time, technician)
assigned_slots = set()


# ========================
# SLOT SELECTION FUNCTION (MODIFIED)
# ========================
def find_single_slot(service_center):
    """
    Returns one recommended slot: (date_str, slot_time, technician)
    This is the FIRST free technician on the EARLIEST available date
    that hasn't been assigned yet.
    """
    df = appt_df[appt_df[APPT_SERVICE_CENTER_COL] == service_center].copy()
    if df.empty:
        return None

    df = df.sort_values([APPT_DATE_COL, APPT_TIME_COL])

    for _, row in df.iterrows():
        date_str = row[APPT_DATE_COL].date().isoformat()
        time_val = row[APPT_TIME_COL]

        for tech in TECHNICIAN_COLS:
            val = str(row[tech]).strip().lower()
            if val == FREE_VALUE:
                tech_name = tech.replace("_", " ")

                # Create unique slot identifier
                slot_key = (service_center, date_str, time_val, tech_name)

                # Check if this slot has already been assigned
                if slot_key not in assigned_slots:
                    # Mark this slot as assigned
                    assigned_slots.add(slot_key)
                    return (date_str, time_val, tech_name)

    return None


# ========================
# STEP 4: ASSIGN SLOTS + WHATSAPP MSGS
# ========================
if "whatsapp_response_2" not in owner_df.columns:
    owner_df["whatsapp_response_2"] = ""

for vid in vehicle_ids:
    owner_mask = owner_df[OWNER_VEHICLE_ID_COL] == vid
    if not owner_mask.any():
        print(f"vehicle_id {vid} not found in Owner Details.")
        continue

    service_center = owner_df.loc[owner_mask, OWNER_SERVICE_CENTER_COL].iloc[0]

    slot = find_single_slot(service_center)
    if slot is None:
        print(f"No free technician found for service_center: {service_center}")
        continue

    date_str, slot_time, tech_name = slot

    # write into owner sheet
    owner_df.loc[owner_mask, NEW_DATE_COL] = date_str
    owner_df.loc[owner_mask, NEW_TIME_COL] = slot_time
    owner_df.loc[owner_mask, NEW_TECH_COL] = tech_name

    # create WhatsApp response 2
    msg = (
        f"Hi, this is the service team from {service_center} regarding your vehicle {vid}.\n\n"
        f"We recommend the following maintenance slot:\n\n"
        f"{date_str} at {slot_time} with {tech_name}\n\n"
        "Reply YES to confirm, or NO if you'd like alternate timings."
    )

    owner_df.loc[owner_mask, "whatsapp_response_2"] = msg

print(f"\nTotal slots assigned: {len(assigned_slots)}")

# ========================
# STEP 5: SAVE OUTPUT
# ========================
output = "updated_owner_assignments.xlsx"

with pd.ExcelWriter(output, engine="openpyxl") as writer:
    failure_df.to_excel(writer, "Failiure Prediction", index=False)
    owner_df.to_excel(writer, "Owner Details", index=False)
    appt_df.to_excel(writer, "Appointment", index=False)
    maint_df.to_excel(writer, "Maintainence Records", index=False)

print("Done. Downloading file...")
files.download(output)

Upload your Excel file:


Saving output_complete_file.xlsx to output_complete_file (7).xlsx

Total slots assigned: 389


/tmp/ipython-input-3017437139.py:150: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  failure_df.to_excel(writer, "Failiure Prediction", index=False)
/tmp/ipython-input-3017437139.py:151: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  owner_df.to_excel(writer, "Owner Details", index=False)
/tmp/ipython-input-3017437139.py:152: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  appt_df.to_excel(writer, "Appointment", index=False)
/tmp/ipython-input-3017437139.py:153: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  maint_df.to_excel(writer, "Maintainence Records", index=False)


Done. Downloading file...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>